<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FT_DSV3d_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ScienceOne-AI/DeepSeek-671B-SFT-Guide.git

## fine tuning deepseek distiller models

In [ ]:
!nvidia-smi

Tue Aug 26 04:19:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             51W /  400W |   29261MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!pip install trl -q

In [ ]:
!pip show trl

Name: trl
Version: 0.21.0
Summary: Train transformer language models with reinforcement learning.
Home-page: https://github.com/huggingface/trl
Author: Leandro von Werra
Author-email: leandro.vonwerra@gmail.com
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: accelerate, datasets, transformers
Required-by: 


In [ ]:
import trl

In [ ]:
from huggingface_hub import HfApi

# Initialize the Hugging Face API client
api = HfApi()

# Search for models containing "deepseek" and "distill"
# The filter argument is a list of strings to search for in model metadata.
# 'deepseek' targets models by the deepseek-ai organization and others
# 'distill' specifically finds the distilled versions
distill_models = api.list_models(
    search="deepseek-ai/DeepSeek-R1-Distill",
    limit=50,  # You can adjust this limit as needed
    full=False # Set to True for more details
)

# Print the name of each model found
print("Found DeepSeek Distilled Models:")
for model in distill_models:
    print(f"- {model.modelId}")

Found DeepSeek Distilled Models:
- deepseek-ai/DeepSeek-V3.1-Base
- deepseek-ai/DeepSeek-V3.1
- deepseek-ai/DeepSeek-R1
- deepseek-ai/DeepSeek-V3-0324
- deepseek-ai/DeepSeek-R1-0528-Qwen3-8B
- deepseek-ai/DeepSeek-R1-0528
- deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
- deepseek-ai/DeepSeek-R1-Distill-Qwen-7B
- bartowski/deepseek-ai_DeepSeek-V3.1-Base-Q4_K_M-GGUF
- deepseek-ai/DeepSeek-V3
- bartowski/deepseek-ai_DeepSeek-V3.1-GGUF
- deepseek-ai/DeepSeek-Coder-V2-Instruct
- deepseek-ai/DeepSeek-R1-Distill-Qwen-32B
- deepseek-ai/DeepSeek-R1-Distill-Qwen-14B
- deepseek-ai/DeepSeek-V3-Base
- deepseek-ai/DeepSeek-R1-Distill-Llama-8B
- deepseek-ai/DeepSeek-Prover-V2-7B
- deepseek-ai/DeepSeek-V2-Lite
- deepseek-ai/DeepSeek-V2-Lite-Chat
- deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct
- deepseek-ai/DeepSeek-R1-Zero
- deepseek-ai/DeepSeek-R1-Distill-Llama-70B
- bartowski/deepseek-ai_DeepSeek-V3.1-Base-GGUF
- deepseek-ai/deepseek-coder-6.7b-base
- deepseek-ai/deepseek-coder-1.3b-instruct
- deepseek-ai

In [ ]:
from datasets import load_dataset

# Define the correct path to the training data JSON file
dataset_path = '/content/DeepSeek-671B-SFT-Guide/data/train_example.json'

print(f"Loading dataset from {dataset_path}...")
# Load the JSON dataset
dataset = load_dataset('json', data_files=dataset_path)
print("Dataset loaded successfully.")
print(dataset)

# Define a formatting function
def format_chat(example):
    formatted_text = ""
    for message in example['messages']:
        if message['role'] == 'system':
            # Assuming system message sets the context, often not explicitly tokenized in some formats
            # You might adjust this based on the specific system tokenization approach
            formatted_text += f"<|im_start|>system\n{message['content']}<|im_end|>\n"
        elif message['role'] == 'user':
            formatted_text += f"<|im_start|>user\n{message['content']}<|im_end|>\n"
        elif message['role'] == 'assistant':
            # Assume assistant turn should be the target for training, potentially add a loss=True flag handling if needed
            formatted_text += f"<|im_start|>assistant\n{message['content']}<|im_end|>\n"
        # Add other roles if necessary

    # The Qwen tokenizer uses <|im_end|> to mark the end of a turn.
    # We might need a final EOS token depending on the model/tokenizer.
    # Qwen models typically use <|endoftext|> as EOS.
    formatted_text += "<|endoftext|>"
    return {"text": formatted_text}

print("Applying formatting function to the dataset...")
# Apply the formatting function
formatted_dataset = dataset.map(format_chat)

print("Dataset formatted successfully.")
print(formatted_dataset)

# Display an example from the formatted dataset to verify
print("\nExample from the formatted dataset:")
print(formatted_dataset['train'][0]['text'])

Loading dataset from /content/DeepSeek-671B-SFT-Guide/data/train_example.json...
Dataset loaded successfully.
DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 1
    })
})
Applying formatting function to the dataset...
Dataset formatted successfully.
DatasetDict({
    train: Dataset({
        features: ['messages', 'text'],
        num_rows: 1
    })
})

Example from the formatted dataset:
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
9.11和9.2谁更大<|im_end|>
<|im_start|>assistant
<think>
嗯，我现在要比较两个数，9.11和9.2，看看哪一个更大。首先，我应该先回忆一下小数比较的规则。可能的话，先对齐小数点后的位数，然后从高位开始逐位比较。

首先，这两个数都是小数，整数部分都是9，所以整数部分一样。接下来需要比较小数部分，也就是小数点后的数字。这时候，9.11的小数部分是0.11，而9.2的小数部分是0.2。不过，可能这里有个问题，因为它们的位数不同，一个是两位小数，一个是一位小数，可能需要统一位数后再比较。

为了更清楚地比较，我可以把这两个数写成相同的小数位数。比如，把9.2写成9.20，这样它们的小数部分都是两位了。这样，9.11的小数部分是11，而9.20的小数部分是20。这时候，20显然比11大，所以9.20比9.11大，也就是说9.2比9.11大。

不过，我是不是哪里弄错了？比如，有没有可能把小数位数搞混了？比如，9.2其实等于9.20，对吗？因为末尾的零不影响数值大小。所以，比较的时候，把9.2看成9.20，那0.20确实比0.11大。所以9.2

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the Hugging Face model ID
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

print(f"Loading model: {model_id}")
# Load the pre-trained model
model = AutoModelForCausalLM.from_pretrained(model_id)
#use_cache=False`

model.use_cache = False

print("Model loaded successfully.")

print(f"Loading tokenizer: {model_id}")
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
print("Tokenizer loaded successfully.")

print("\nModel and Tokenizer loaded for fine-tuning.")

Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
Model loaded successfully.
Loading tokenizer: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
Tokenizer loaded successfully.

Model and Tokenizer loaded for fine-tuning.


In [ ]:
# After reinstallation, re-attempt to initialize trainer and start training

from transformers import TrainingArguments
from trl import SFTTrainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/sft_trainer_output",  # Output directory for checkpoints and logs
    num_train_epochs=10,                 # Number of training epochs
    per_device_train_batch_size=4,      # Batch size per GPU
    learning_rate=2e-5,                 # Learning rate
    bf16=True,                          # Enable bfloat16 training (L4 supports it)
    logging_steps=1,                   # Log every 10 steps
    save_steps=1,                     # Save checkpoint every 100 steps
    save_total_limit=2,                 # Keep at most 2 checkpoints
    gradient_accumulation_steps=2,      # Accumulate gradients over 2 steps
    gradient_checkpointing=True,        # Enable gradient checkpointing
    report_to="none",                   # Disable reporting to external platforms
    push_to_hub=False,                  # Do not push to Hugging Face Hub
    remove_unused_columns=False,        # Keep unused columns in dataset
)

# Check if tokenizer has a pad token, if not, set it to eos_token
# Assuming 'tokenizer' and 'formatted_dataset' are available from previous successful steps
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Tokenizer pad_token set to eos_token: {tokenizer.pad_token}")

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset['train'], # Use the 'train' split of the dataset
    args=training_args,                   # Pass the training arguments
)

print("\nTraining arguments and SFTTrainer re-initialized.")
print("Starting the fine-tuning process...")
# Start training
trainer.train()
print("Fine-tuning process initiated.")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.



Training arguments and SFTTrainer re-initialized.
Starting the fine-tuning process...


Step,Training Loss
1,1.115400
2,0.789800
3,0.451700
4,0.345400
5,0.146900
6,0.126900
7,0.109600
8,0.103300
9,0.097600
10,0.093600


Fine-tuning process initiated.
